In [ ]:
# install required libraries
!pip install cfgrib
!pip install xarray

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import xarray as xr
import cfgrib

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# The data was gathered from https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=overview

In [ ]:
# Load the files
weather1_path = '/content/drive/MyDrive/0.Data science final project/Weather Data/weather1.grib'
weather2_path = '/content/drive/My Drive/0.Data science final project/Weather Data/weather2.grib'

In [ ]:
# Open the files
weather1 = xr.open_dataset(weather1_path, engine='cfgrib')

ERROR:cfgrib.dataset:skipping variable: paramId==228 shortName='tp'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1709251200, 1709254800, 1709258400, ..., 1735678800, 1735682400,
       1735686000])) new_value=Variable(dimensions=('time',), data=array([1709229600, 1709272800, 1709316000, 1709359200, 1709402400,
       1709445600, 1709488800, 1709532000, 1709575200, 1709618400,
       1709661600, 1709704800, 1709748000, 1709791200, 1709834400,
       1709877600, 1709920800, 1709964000, 1710007200, 1710050400,
       1710093600, 1710136800, 1710180000, 1710223200, 1710266400,
       1710309600, 1710352800, 

In [ ]:
weather2 = xr.open_dataset(weather2_path, engine='cfgrib')

ERROR:cfgrib.dataset:skipping variable: paramId==228 shortName='tp'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/usr/local/lib/python3.11/dist-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1735689600, 1735693200, 1735696800, ..., 1741860000, 1741863600,
       1741867200])) new_value=Variable(dimensions=('time',), data=array([1735668000, 1735711200, 1735754400, 1735797600, 1735840800,
       1735884000, 1735927200, 1735970400, 1736013600, 1736056800,
       1736100000, 1736143200, 1736186400, 1736229600, 1736272800,
       1736316000, 1736359200, 1736402400, 1736445600, 1736488800,
       1736532000, 1736575200, 1736618400, 1736661600, 1736704800,
       1736748000, 1736791200, 

In [ ]:
# concatenate along the time dimension
weather = xr.concat([weather1, weather2], dim='time')

In [ ]:
print(weather)

<xarray.Dataset> Size: 338MB
Dimensions:     (time: 9061, latitude: 69, longitude: 45)
Coordinates:
    number      int64 8B 0
  * time        (time) datetime64[ns] 72kB 2024-03-01 ... 2025-03-13T12:00:00
    step        timedelta64[ns] 8B 00:00:00
    surface     float64 8B 0.0
  * latitude    (latitude) float64 552B 49.0 48.75 48.5 ... 32.5 32.25 32.0
  * longitude   (longitude) float64 360B -125.0 -124.8 -124.5 ... -114.2 -114.0
    valid_time  (time) datetime64[ns] 72kB 2024-03-01 ... 2025-03-13T12:00:00
Data variables:
    u10         (time, latitude, longitude) float32 113MB -0.9383 ... -2.046
    v10         (time, latitude, longitude) float32 113MB 2.149 2.273 ... 2.394
    t2m         (time, latitude, longitude) float32 113MB 275.7 274.4 ... 285.4
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    insti

In [ ]:
# convert x array into a pandas dataframe
weather_data_raw = weather.to_dataframe()


In [ ]:
print(weather_data_raw)

                                             u10       v10         t2m  \
time                latitude longitude                                   
2024-03-01 00:00:00 49.0     -125.00   -0.938263  2.148621  275.652100   
                             -124.75   -0.049591  2.272644  274.384521   
                             -124.50    0.451385  2.405457  273.480225   
                             -124.25   -0.449982  2.423035  274.294678   
                             -124.00   -1.340607  2.299011  276.157959   
...                                          ...       ...         ...   
2025-03-13 12:00:00 32.0     -115.00    3.087631  0.372604  287.070557   
                             -114.75    2.951889 -0.528763  286.365479   
                             -114.50    1.884506 -0.262161  285.539307   
                             -114.25   -1.259048  1.173386  284.863525   
                             -114.00   -2.046158  2.394089  285.404541   

                                     

In [ ]:
# Clean up data
u_wind = weather_data_raw["u10"]
v_wind = weather_data_raw["v10"]
wind_speed = np.sqrt(u_wind**2 + v_wind**2)
wind_direction_rad = np.arctan2(v_wind, u_wind)
wind_direction_deg = np.degrees(wind_direction_rad)
wind_direction_deg = (wind_direction_deg + 360) % 360

In [ ]:
# Convert to Pandas DataFrame
weather_df = pd.DataFrame({
    'time': weather_data_raw.index.get_level_values('time').hour,
    'latitude': weather_data_raw.index.get_level_values('latitude'),
    'longitude': weather_data_raw.index.get_level_values('longitude'),
    'eastward_wind': u_wind.values.flatten(),
    'northward_wind': v_wind.values.flatten(),
    'temperature': weather_data_raw['t2m'].values.flatten(),
    'valid_time': weather_data_raw['valid_time'].values,
})


In [ ]:
weather_df.head()

,time,latitude,longitude,eastward_wind,northward_wind,temperature,valid_time
28134400,12,32.0,-115.00,3.087631,0.372604,287.070557,2025-03-13 12:00:00
28134401,12,32.0,-114.75,2.951889,-0.528763,286.365479,2025-03-13 12:00:00
28134402,12,32.0,-114.50,1.884506,-0.262161,285.539307,2025-03-13 12:00:00
28134403,12,32.0,-114.25,-1.259048,1.173386,284.863525,2025-03-13 12:00:00
28134404,12,32.0,-114.00,-2.046158,2.394089,285.404541,2025-03-13 12:00:00


In [ ]:
weather_df.describe()

,time,latitude,longitude,eastward_wind,northward_wind,temperature,valid_time
count,2.813440e+07,2.813440e+07,2.813440e+07,2.813440e+07,2.813440e+07,2.813440e+07,28134405
mean,1.149211e+01,4.050000e+01,-1.195000e+02,1.108943e+00,-6.322211e-01,2.852341e+02,2024-09-05 17:59:59.999999232
min,0.000000e+00,3.200000e+01,-1.250000e+02,-2.167062e+01,-1.845952e+01,2.412347e+02,2024-03-01 00:00:00
25%,5.000000e+00,3.625000e+01,-1.222500e+02,-4.512177e-01,-1.779373e+00,2.785115e+02,2024-06-03 09:00:00
50%,1.100000e+01,4.050000e+01,-1.195000e+02,8.752136e-01,-1.631470e-01,2.857217e+02,2024-09-05 18:00:00
75%,1.700000e+01,4.475000e+01,-1.167500e+02,2.349106e+00,1.194931e+00,2.904680e+02,2024-12-09 03:00:00
max,2.300000e+01,4.900000e+01,-1.140000e+02,2.014445e+01,2.459706e+01,3.238103e+02,2025-03-13 12:00:00
std,6.921802e+00,4.979123e+00,3.246793e+00,2.243590e+00,3.210492e+00,2.198436e+01,NaN


In [ ]:
# Clean up DataFrame

# Sort by date and time
weather_df = weather_df.sort_values(by='time')

# Reset the index
weather_df = weather_df.reset_index(drop=True)

# Display the cleaned and sorted DataFrame
weather_df.head()

,time,latitude,longitude,eastward_wind,northward_wind,temperature,valid_time
0,0,32.0,-118.25,4.547577,-6.464828,290.576416,2024-11-04
1,0,32.0,-118.50,5.390350,-7.320297,290.330322,2024-11-04
2,0,32.0,-118.75,6.355194,-8.207993,290.133057,2024-11-04
3,0,32.0,-119.00,6.975311,-8.845688,289.976807,2024-11-04
4,0,32.0,-119.25,7.395233,-9.300766,289.736572,2024-11-04


In [ ]:
# put valid time as the first row
weather_df = weather_df.reindex(columns=['valid_time'] + [col for col in weather_df.columns if col != 'valid_time'])

In [ ]:
weather_df.head()

,date,time,latitude,longitude,eastward_wind,northward_wind,temperature
65159,2024-03-01,0,48.75,-116.50,0.329315,2.819519,273.657959
521554,2024-03-01,0,48.75,-124.50,-0.557404,2.734558,274.109131
521558,2024-03-01,0,48.75,-122.50,-1.852325,3.740417,277.519287
521559,2024-03-01,0,48.75,-122.75,-3.162872,5.892761,277.972412
521560,2024-03-01,0,48.75,-123.00,-3.583771,6.359558,278.228271


In [ ]:
# rename valid_time column to date
weather_df = weather_df.rename(columns={'valid_time': 'date'})

In [ ]:
# Sort values by date first and then by time
weather_df = weather_df.sort_values(by=['date', 'time'])

In [ ]:
# Save to a csv
weather_df.to_csv('/content/drive/MyDrive/0.Data science final project/Weather Data/weather_data.csv', index=False)